In [116]:
import pandas as pd 
import numpy as np
from pandas import pivot_table
train = pd.read_csv("data/FakeNews_Training_Set.csv", header=0, delimiter=",")
test = pd.read_csv("data/FakeNews_Test_Set.csv", header=0, delimiter=",")
label_mapping = {
           'REAL': 1,
           'FAKE': 0}

train['label'] = train['label'].map(label_mapping)
test['label'] = test['label'].map(label_mapping)

In [117]:
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer,sent_tokenize,word_tokenize
import re

In [118]:
def process_content(sentListTrain):
    try:
        tokenized_sents = [word_tokenize(i) for i in sentListTrain]
        for i in tokenized_sents:
            tagged=nltk.pos_tag(i)
            mytaggedList.insert(count+1,tagged)            

    except Exception as e:
        print(str(e))
    return mytaggedList

In [119]:
def pivot_postags(modelDF):

    overall_pos_tags = pd.DataFrame(['postag','totcount','sno'])

    for index, row in modelDF.iterrows():
        if index < 50:
            twlist=[]
            sentListTrain =  sent_tokenize(row['text'])
            for sent in sentListTrain:
                #print (sent)
                #recvListTrain = process_content(sent)
                words= word_tokenize(sent)
                recvListTrain=nltk.pos_tag(words)
                for wlist in recvListTrain:
                    twlist.append(wlist[1])


            df = pd.DataFrame({'postag': twlist})
            counts_df = pd.DataFrame(df.groupby('postag').size().rename('totcount'))
            counts_df['sno']=row['sno']

            #overall_pos_tags.append(counts_df,ignore_index=False)

            #print (counts_df)

            frames = [counts_df, overall_pos_tags]
            overall_pos_tags = pd.concat(frames)

    overall_pos_tags['postags'] = overall_pos_tags.index
    pivot_postags=overall_pos_tags.pivot(index='sno', columns='postags', values='totcount').fillna(value=0)
    pivot_postags['sno'] = pivot_postags.index
    postags_merged=pivot_postags.join(train[['sno','label']], on='sno', how='inner', rsuffix='_1', sort=False)

    return postags_merged

In [122]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run


print ('Training data to Extract and Pivot')

postags_merged = pivot_postags(train)
#postags_merged = postags_merged.drop(['sno', 'sno_1'], axis=1)

print ('Training model')

print ( postags_merged )

forest = forest.fit( postags_merged.drop(['label','sno','sno_1'], axis=1), postags_merged["label"] )

print ('Tesing data to Extract and Pivot')

test_postags_merged = pivot_postags(test)
#test_postags_merged = test_postags_merged.drop(['sno', 'sno_1'], axis=1)

print ('Model Prediction started')

print ( test_postags_merged )

# Use the random forest to make sentiment label predictions
result = forest.predict(test_postags_merged.drop(['label', 'sno','sno_1'], axis=1))



Training data to Extract and Pivot


C:\Users\abira\Anaconda3\lib\site-packages\pandas\indexes\api.py:71: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


Training model
          0    1    2    #     $    ''    (    )      ,      .  ...     VBP  \
sno                                                             ...           
51.0    0.0  0.0  0.0  0.0   3.0   0.0  1.0  1.0   13.0   19.0  ...     2.0   
71.0    0.0  0.0  0.0  0.0   0.0  19.0  0.0  0.0   52.0   35.0  ...    14.0   
95.0    0.0  0.0  0.0  0.0   0.0   0.0  1.0  1.0    6.0    5.0  ...     0.0   
127.0   0.0  0.0  0.0  0.0   0.0  18.0  2.0  2.0   51.0   54.0  ...    17.0   
139.0   0.0  0.0  0.0  1.0   0.0   1.0  0.0  0.0   65.0   50.0  ...    19.0   
143.0   0.0  0.0  0.0  0.0   0.0   5.0  0.0  0.0   38.0   42.0  ...    33.0   
172.0   0.0  0.0  0.0  0.0   0.0   0.0  1.0  1.0    2.0    1.0  ...     1.0   
186.0   0.0  0.0  0.0  0.0   0.0   9.0  0.0  0.0   24.0   21.0  ...    15.0   
384.0   0.0  0.0  0.0  0.0   0.0   0.0  0.0  0.0    0.0    1.0  ...     0.0   
421.0   0.0  0.0  0.0  0.0   0.0   6.0  1.0  1.0   13.0   17.0  ...     3.0   
431.0   0.0  0.0  0.0  0.0   0.0  11.

C:\Users\abira\Anaconda3\lib\site-packages\pandas\indexes\api.py:71: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


Model Prediction started
          0    1    2    #    $    ''    (    )     ,     .  ...     VBP  \
sno                                                          ...           
470.0   0.0  0.0  0.0  0.0  0.0  15.0  4.0  4.0  43.0  36.0  ...    36.0   
480.0   0.0  0.0  0.0  0.0  0.0   3.0  0.0  0.0  22.0  19.0  ...     6.0   
557.0   0.0  0.0  0.0  0.0  0.0   5.0  2.0  2.0  49.0  52.0  ...    18.0   
629.0   0.0  0.0  0.0  0.0  0.0   1.0  0.0  0.0   1.0   1.0  ...     0.0   
642.0   0.0  0.0  0.0  0.0  0.0   1.0  0.0  0.0   1.0   1.0  ...     0.0   
660.0   0.0  0.0  0.0  0.0  0.0   0.0  3.0  3.0  94.0  67.0  ...    40.0   
1243.0  0.0  0.0  0.0  0.0  0.0  17.0  1.0  1.0  33.0  31.0  ...     9.0   
1442.0  0.0  0.0  0.0  0.0  0.0   1.0  0.0  0.0   3.0   4.0  ...     2.0   
1928.0  0.0  0.0  0.0  0.0  0.0  10.0  0.0  0.0  52.0  37.0  ...    13.0   
2165.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0   2.0   1.0  ...     0.0   
2491.0  0.0  0.0  0.0  0.0  1.0   0.0  1.0  1.0  49.0  55.0  ..

In [121]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(test_postags_merged['label'], result)
accuracy_score(test_postags_merged['label'], result)

0.55000000000000004